In [1]:
from langchain.llms import OpenAI
from langchain import PromptTemplate, LLMChain
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
import sentence_transformers
import os

os.environ["OPENAI_API_KEY"] = "EMPTY"
os.environ["OPENAI_API_BASE"] = os.environ["LLM_API_SERVER_BASE"]
model = os.environ["LLM_LOADED"]

In [50]:
llm = OpenAI(model=model,logit_bias=None, max_tokens=30, temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

In [3]:
model_name = "intfloat/e5-large-v2"

hf = HuggingFaceEmbeddings(model_name=model_name)

In [4]:
#loader = PyPDFLoader("../data/iceberg-overview.pdf")
loader = PyPDFLoader("../data/PT-Bank-Central-Asia-Tbk.pdf")
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(pages)

In [5]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [6]:
embedding = hf 
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory='db'
                                 )

retriever = vectordb.as_retriever()

In [57]:
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Answer in a short and concise manner
{context}

Question: {question}"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [58]:
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, chain_type_kwargs=chain_type_kwargs)

In [59]:
%%time
#query = "What is apache iceberg?"
query = "How big is BCA assset?"
llm_response = qa.run(query)



Answer: Rp 1.029,83 Trillion (USD 68,5 Billion)
CPU times: user 32.5 s, sys: 163 ms, total: 32.6 s
Wall time: 3.4 s
